In [1]:
import numpy as np
import pandas as pd
from google.colab import drive
from google import colab

In [2]:
prefix = '/content/drive'
from google.colab import drive
drive.mount(prefix, force_remount=True)

Mounted at /content/drive


In [122]:
tracks_path = '/content/drive/My Drive/Academics/Sophomore Fall/CIS 5500/Data/tracks.csv'
artists_path = '/content/drive/My Drive/Academics/Sophomore Fall/CIS 5500/Data/artists.csv'
save_path = '/content/drive/My Drive/Academics/Sophomore Fall/CIS 5500/Processed Data/'

In [123]:
tracks = pd.read_csv(tracks_path)
artists = pd.read_csv(artists_path)

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (2,4,6) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [124]:
# Drop unneeded columns
tracks.drop(['disc_number', 'track_number', 'time_signature', 'year'], inplace=True, axis=1)
artists.drop(['ambiguous_artist', 'mbid', 'scrobbles_lastfm'], inplace=True, axis=1)

In [125]:
# Use artist_mb as artist_name
artists.rename(columns={'artist_mb': 'artist_name', 'listeners_lastfm': 'listeners'}, inplace=True);
artists.dropna(subset=['artist_name'], inplace=True);
artists.drop(['artist_lastfm'], axis=1, inplace=True);

In [126]:
# Merge country data from MusicBrainz and LastFM
artists['country'] = artists['country_mb']
artists.loc[artists['country'].isnull(), 'country'] = artists.loc[artists['country'].isnull(), 'country_lastfm']
artists.drop(['country_mb', 'country_lastfm'], axis=1, inplace=True);

In [127]:
# Merge genre data from MusicBrainz and LastFM
artists['genres'] = artists['tags_mb']
artists.loc[artists['genres'].isnull(), 'genres'] = artists.loc[artists['genres'].isnull(), 'tags_lastfm']
artists.drop(['tags_mb', 'tags_lastfm'], axis=1, inplace=True);

In [128]:
# Set default values for listeners and genres
artists['listeners'] = artists['listeners'].fillna(0).astype(int)
artists['genres'] = artists['genres'].fillna('')

In [129]:
# Split strings for artist names and ids into lists
tracks['artists'] = tracks['artists'].apply(lambda x: x.strip("[]'").split(', '))
tracks['artist_ids'] = tracks['artist_ids'].apply(lambda x: x.strip("[]'").split(', '))

In [130]:
# Check for artist name and id mismatch, remove these rows
t1 = tracks['artists'].str.len()
t2 = tracks['artist_ids'].str.len()
mismatch = (t1 != t2)
tracks = tracks[~mismatch]

In [131]:
# Map artist name to Spotify id, used later to replace MusicBrainz id in artists with Spotify id
artist_name_to_id = tracks[['artists', 'artist_ids']]
artist_name_to_id = artist_name_to_id.explode(['artists', 'artist_ids'])
artist_name_to_id['artists'] = artist_name_to_id['artists'].apply(lambda x: x.strip("'"))
artist_name_to_id['artist_ids'] = artist_name_to_id['artist_ids'].apply(lambda x: x.strip("'"))
artist_name_to_id = artist_name_to_id.drop_duplicates()

In [132]:
# Associate artist name with Spotify id
m = artist_name_to_id.set_index('artists').to_dict()['artist_ids']
artists['artist_id'] = artists['artist_name'].apply(lambda x: m[x] if x in m else np.nan)
artists.dropna(subset=['artist_id'], inplace=True)
artists = artists[['artist_id', 'artist_name', 'listeners', 'country', 'genres']]

In [133]:
# Drop artist names from tracks, explode ids
tracks.drop(['artists'], axis=1, inplace=True)
tracks = tracks.explode('artist_ids')

In [134]:
# Take subtables from tracks to form SongBy and AlbumBy
song_by = tracks[['id', 'artist_ids']].copy()
song_by.rename(columns={'id': 'track_id', 'artist_ids': 'artist_id'}, inplace=True)
album_by = tracks[['album_id', 'artist_ids']].copy().drop_duplicates()
album_by.rename(columns={'artist_ids': 'artist_id'}, inplace=True)

In [135]:
# Form album table from tracks
albums = tracks[['album_id', 'album']].copy().drop_duplicates()
albums.rename(columns={'album': 'album_name'}, inplace=True)

In [136]:
# Form genres table from artists
genres = artists[['artist_id', 'genres']].copy()
genres['genres'] = genres['genres'].apply(lambda x: x.split('; '))
genres = genres.explode('genres')
genres.rename(columns={'genres': 'genre'}, inplace=True)
genres.replace('', np.nan, inplace=True)
genres.dropna(subset=['genre'], inplace=True)
artists.drop(['genres'], axis=1, inplace=True)

In [137]:
# Rename columns in tracks
tracks.rename(columns={'id': 'track_id', 'name': 'track_name', 'key': 'music_key'}, inplace=True)
tracks.drop(['artist_ids', 'album'], axis=1, inplace=True)

In [138]:
# Drop duplicate entries
tracks = tracks.drop_duplicates(subset=['track_id']) 
artists = artists.drop_duplicates(subset=['artist_id']) 
albums = albums.drop_duplicates(subset=['album_id']) 
genres = genres.drop_duplicates()
song_by = song_by.drop_duplicates()
album_by = album_by.drop_duplicates()

In [139]:
# Save final dataframes as csv
tracks.to_csv(save_path + 'Songs.csv')
artists.to_csv(save_path + 'Artists.csv')
albums.to_csv(save_path + 'Albums.csv')
genres.to_csv(save_path + 'Genres.csv')
song_by.to_csv(save_path + 'SongBy.csv')
album_by.to_csv(save_path + 'AlbumBy.csv')